## Lesson 3: Binary format modification

**Objectives**: learn about resource views; find specific children in a resource tree and modify them; manipulate ELF program headers

In this lesson, we're going to find the LOAD program header, and change its permission to make it non-executable.

In [1]:
from ofrak import OFRAK
from ofrak_tutorial.helper_functions import create_hello_world_binary

create_hello_world_binary()

ofrak = OFRAK()
basic_context = await ofrak.create_ofrak_context()
root_resource = await basic_context.create_root_resource_from_file("hello_world")
unpack_result = await root_resource.unpack_recursively()

Using OFRAK Community License.


So, after unpacking, our root resource is an ELF resource, right?...

In [2]:
for tag in sorted(root_resource.get_tags(), key=str):
    print(tag)

Elf
File
FilesystemEntry
GenericBinary
LinkableBinary
Program


Yes... But it's also all of the above. What if we want to run ELF-specific methods on it? This is what **resource views** are designed for: they're a way to, as their name suggests, *view* a resource as a particular type for the purpose of executing operations specific to that type.

We can't call the ELF-specific method `get_header` on a generic resource...

In [3]:
# Will fail with an AttributeError!
elf_header = root_resource.get_header()

AttributeError: 'Resource' object has no attribute 'get_header'

... So we create an *ELF view* first to access this ELF-specific functionality:

In [4]:
from ofrak.core import Elf

elf_view = await root_resource.view_as(Elf)

elf_header = await elf_view.get_header()

And how do we get back the resource from a resource view? By using the `resource` attribute of any resource view.

Trying to get the children of an ELF view will fail...

In [5]:
# Will fail with an AttributeError!
await elf_view.get_children()

AttributeError: 'Elf' object has no attribute 'get_children'

... That's what the `resource` attribute is for:

In [6]:
list(await elf_view.resource.get_children())[0]

Resource(resource_id=bd10b0f7bae3435faa84b0034ccda0e0, tag=[ElfBasicHeader], data=bd10b0f7bae3435faa84b0034ccda0e0)

So we're going to use this ELF view for all ELF-related functionality.

How would we find the LOAD program header?

In [7]:
from ofrak_type.memory_permissions import MemoryPermissions
from ofrak.core import ElfProgramHeader, ElfProgramHeaderType


async def get_exec_load_program_header(elf_view: Elf) -> ElfProgramHeader:
    """Return the first executable LOAD program header in `elf_view`."""
    for program_header in await elf_view.get_program_headers():
        if (
            program_header.p_type == ElfProgramHeaderType.LOAD.value
            and program_header.p_flags & MemoryPermissions.X.value
        ):
            return program_header
    raise RuntimeError(f"Could not find executable LOAD program header in {elf_view}")


exec_load_program_header = await get_exec_load_program_header(elf_view)
print(exec_load_program_header)

ElfProgramHeader(segment_index=3, p_type=1, p_offset=4096, p_vaddr=4198400, p_paddr=4198400, p_filesz=329, p_memsz=329, p_flags=5, p_align=4096)


Now that we know how to find it, let's see how we can modify it!

In [8]:
from ofrak.core import ElfProgramHeaderModifier, ElfProgramHeaderModifierConfig


async def make_program_header_noexec(program_header):
    await program_header.resource.run(
        ElfProgramHeaderModifier,
        ElfProgramHeaderModifierConfig(p_flags=program_header.p_flags & ~MemoryPermissions.X.value),
    )


print(f"still executable: p_flags={exec_load_program_header.p_flags}")
await make_program_header_noexec(exec_load_program_header)
print(f"modified: p_flags={exec_load_program_header.p_flags}")

still executable: p_flags=5
modified: p_flags=4


Let's put it all together now. As a reminder, here's the typical OFRAK workflow we were mentioning in [Lesson 2](2_ofrak_internals.ipynb):

- create an OFRAK resource from something, typically a file on disk
  + **unpack** the resource
    - **modify** the resource
  + re-**pack** the resource
- export the modified and repacked resource, typically to a file on disk

See if you can spot it in the code below.

In [9]:
async def make_elf_load_header_noexec(ofrak_context, input_filename, output_filename):
    root_resource = await ofrak_context.create_root_resource_from_file(input_filename)
    await root_resource.unpack_recursively()

    elf_view = await root_resource.view_as(Elf)
    exec_load_program_header = await get_exec_load_program_header(elf_view)
    await make_program_header_noexec(exec_load_program_header)

    await root_resource.pack()
    await root_resource.flush_data_to_disk(output_filename)


await make_elf_load_header_noexec(basic_context, "hello_world", "nohello_world")

In [10]:
# Expecting a segmentation fault...
!chmod +x nohello_world && ./nohello_world

Segmentation fault (core dumped)


[Next page](4_simple_code_modification.ipynb)